In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt

os.chdir('/Users/Evan/Kaggle/')

In [2]:
user_info_test = pd.read_csv('./Credit/test/user_info_test.txt',names=['ID', 'gender','career','education', 'marriage', 'hukou'],header=None)
loan_time_test = pd.read_csv('./Credit/test/loan_time_test.txt',names=['ID', 'timestamp_money'], header=None)
usersID_test = pd.read_csv('./Credit/test/usersID_test.txt',names=['ID','Label'],header=None)
bank_detail_test = pd.read_csv('./Credit/test/bank_detail_test.txt',names=['ID', 'timestamp_bank', 'type', 'money','salary'],header=None)
browse_history_test = pd.read_csv('./Credit/test/browse_history_test.txt',names=['ID', 'timestamp_browse', 'browse_data', 'browser_code'],header=None)
bill_detail_test = pd.read_csv('./Credit/test/bill_detail_test.txt',names=['ID', 'timestamp_bill','bank_id', 'last_bill', 'last_repayment', 'credit_line', 'current_balance', 'minimum_payments', 'amount_transactions', 'current_money', 'adjust_money','cycle_interest','available_money','cash_advance','repayment_status'],header=None)

In [3]:
data_user_test = pd.merge(user_info_test,pd.merge(loan_time_test,usersID_test,on='ID',how='inner'),on='ID',how='inner')
data_user_test = data_user_test.apply(lambda x: x.astype(str))

In [4]:
data_user_test.duplicated().sum()
data_user_test.isnull().sum()

ID                 0
gender             0
career             0
education          0
marriage           0
hukou              0
timestamp_money    0
Label              0
dtype: int64

In [5]:
# Bank information extraction

# 净收入

bank_detail_test['money_direction'] = bank_detail_test['type'].replace({0:1,1:-1})*bank_detail_test['money']
data_net = bank_detail_test.groupby(['ID'])['money_direction'].agg({'sum','count','mean'}).rename(columns=dict(sum='sum_total',count='freq_total',mean='mean_total'))
data_net['ID'] = data_net.index
data_net.reset_index(drop = True)
data_net['ID'] = data_net['ID'].apply(lambda x:str(x))

# 收入信息汇总

data_income = bank_detail_test[bank_detail_test.type==0].groupby(['ID'])['money'].agg({'sum','count','mean'}).rename(columns=dict(sum='sum_income', count='freq_income',mean='mean_income'))
data_income['ID'] = data_income.index
data_income.reset_index(drop = True)
data_income['ID'] = data_income['ID'].apply(lambda x:str(x))

bank_detail_test.groupby("ID")['ID'].count().to_frame().shape

len(data_income)/len(data_user_test)

# 支出信息汇总

data_spend = bank_detail_test[bank_detail_test.type==1].groupby(['ID'])['money'].agg({'sum','count','mean'}).rename(columns=dict(sum='sum_spend', count='freq_spend',mean='mean_spend'))
data_spend['ID'] = data_spend.index
data_spend.reset_index(drop = True)
data_spend['ID'] = data_spend['ID'].apply(lambda x:str(x))

data_bank = pd.merge(pd.merge(pd.merge(data_user_test,data_net,on='ID',how='left'),data_income,on='ID',how='left'),data_spend,on='ID',how='left')

data_bank.head()

# Browser information extraction

data_browse = browse_history_test.loc[:, ['ID', 'browse_data']].groupby(['ID']).mean()
data_browse.head()

# Bill information extraction

print(bill_detail_test.columns)
print(bank_detail_test.columns)
bill_detail_test.head()

data_bill = bill_detail_test.groupby(['ID'])['credit_line','cash_advance','amount_transactions'].sum()
bill_detail_test.groupby(['ID'])['credit_line','cash_advance','amount_transactions'].sum().head()
bill_detail_test.groupby(['ID'])['credit_line','cash_advance','amount_transactions'].sum().reset_index().head()
bill_detail_test.groupby(['ID'])['credit_line','cash_advance','amount_transactions'].sum().head()
data_bill = bill_detail_test.assign(bill_credit=bill_detail_test.last_bill/bill_detail_test.credit_line).groupby('ID')['bill_credit'].mean().to_frame().replace({np.inf:2}).reset_index().merge(bill_detail_test.groupby(['ID'])['credit_line','cash_advance','amount_transactions'].sum().reset_index(),on='ID',how='outer')

data_bill.head()

data_browser = browse_history_test.groupby(['ID','browser_code']).browse_data.sum().reset_index().pivot(index='ID', columns='browser_code', values='browse_data').fillna(0)

data_browser.columns = ['browser'+ str(x) for x in data_browser.columns]

data_browser.reset_index().head()

# 数据合并

data_bill.ID = data_bill.ID.astype(str)
data_browser = data_browser.reset_index()
data_browser.ID = data_browser.ID.astype(str)

data_test = data_bank.merge(data_bill,on='ID',how='left').merge(data_browser,on='ID',how='left')

print(data_test.shape)

print(data_test.isnull().sum()/len(data_test))

print(data_test.shape)

data_test.to_csv('/Users/Evan/Desktop/data_test.csv')

Index(['ID', 'timestamp_bill', 'bank_id', 'last_bill', 'last_repayment',
       'credit_line', 'current_balance', 'minimum_payments',
       'amount_transactions', 'current_money', 'adjust_money',
       'cycle_interest', 'available_money', 'cash_advance',
       'repayment_status'],
      dtype='object')
Index(['ID', 'timestamp_bank', 'type', 'money', 'salary', 'money_direction'], dtype='object')
(13899, 32)
ID                     0.000000
gender                 0.000000
career                 0.000000
education              0.000000
marriage               0.000000
hukou                  0.000000
timestamp_money        0.000000
Label                  0.000000
freq_total             0.948989
mean_total             0.948989
sum_total              0.948989
freq_income            0.948989
mean_income            0.948989
sum_income             0.948989
freq_spend             0.949277
mean_spend             0.949277
sum_spend              0.949277
bill_credit            0.113605
credit_line

In [6]:
pd.crosstab(data_user_test.gender,data_user_test.Label,margins=True)

Label,nan,All
gender,,
0,438,438
1,10829,10829
2,2632,2632
All,13899,13899


In [7]:
pd.crosstab(data_user_test.career,data_user_test.Label).apply(lambda x: x/x.sum(), axis=1)

Label,nan
career,
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [8]:
pd.crosstab(data_user_test.gender,data_user_test.Label).apply(lambda x: x/x.sum(), axis=1)

Label,nan
gender,
0,1.0
1,1.0
2,1.0


In [9]:
pd.crosstab(data_user_test.education,data_user_test.Label).apply(lambda x: x/x.sum(), axis=1)

Label,nan
education,
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [10]:
pd.crosstab(data_user_test.marriage,data_user_test.Label).apply(lambda x: x/x.sum(), axis=1)

Label,nan
marriage,
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0


In [11]:
pd.crosstab(data_user_test.hukou,data_user_test.Label).apply(lambda x: x/x.sum(), axis=1)

Label,nan
hukou,
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


# Bank information extraction

## 净收入

In [12]:
bank_detail_test['money_direction'] = bank_detail_test['type'].replace({0:1,1:-1})*bank_detail_test['money']
data_net = bank_detail_test.groupby(['ID'])['money_direction'].agg({'sum','count','mean'}).rename(columns=dict(sum='sum_total',count='freq_total',mean='mean_total'))
data_net['ID'] = data_net.index
data_net.reset_index(drop = True)
data_net['ID'] = data_net['ID'].apply(lambda x:str(x))

## 收入信息汇总

In [13]:
data_income = bank_detail_test[bank_detail_test.type==0].groupby(['ID'])['money'].agg({'sum','count','mean'}).rename(columns=dict(sum='sum_income', count='freq_income',mean='mean_income'))
data_income['ID'] = data_income.index
data_income.reset_index(drop = True)
data_income['ID'] = data_income['ID'].apply(lambda x:str(x))

bank_detail_test.groupby("ID")['ID'].count().to_frame().shape

len(data_income)/len(data_user_test)

0.05101086409094179

## 支出信息汇总

In [14]:
data_spend = bank_detail_test[bank_detail_test.type==1].groupby(['ID'])['money'].agg({'sum','count','mean'}).rename(columns=dict(sum='sum_spend', count='freq_spend',mean='mean_spend'))
data_spend['ID'] = data_spend.index
data_spend.reset_index(drop = True)
data_spend['ID'] = data_spend['ID'].apply(lambda x:str(x))

data_bank = pd.merge(pd.merge(pd.merge(data_user_test,data_net,on='ID',how='left'),data_income,on='ID',how='left'),data_spend,on='ID',how='left')
data_bank.head()

,ID,gender,career,education,marriage,hukou,timestamp_money,Label,freq_total,mean_total,sum_total,freq_income,mean_income,sum_income,freq_spend,mean_spend,sum_spend
0,57189,1,2,4,3,2,5932395087,nan,478.0,-6.126388,-2928.413552,109.0,14.615321,1593.070037,369.0,12.253343,4521.483589
1,59778,1,2,3,1,4,5932827087,nan,618.0,-5.286107,-3266.814389,154.0,13.207106,2033.894275,464.0,11.423941,5300.708664
2,59779,1,2,4,1,2,5932827087,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64690,1,2,4,2,1,5933518287,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63295,1,2,3,2,2,5933345487,nan,1005.0,-1.506225,-1513.756155,424.0,12.843720,5445.737430,581.0,11.978474,6959.493585


# Browser information extraction

In [15]:
data_browse = browse_history_test.loc[:, ['ID', 'browse_data']].groupby(['ID']).mean()
data_browse.head()

,browse_data
ID,
55597,113.937500
55598,117.193750
55601,119.343511
55602,112.666667
55603,118.277778


## Bill information extraction

In [16]:
print(bill_detail_test.columns)
print(bank_detail_test.columns)
bill_detail_test.head()

data_bill = bill_detail_test.groupby(['ID'])['credit_line','cash_advance','amount_transactions'].sum()
bill_detail_test.groupby(['ID'])['credit_line','cash_advance','amount_transactions'].sum().head()
bill_detail_test.groupby(['ID'])['credit_line','cash_advance','amount_transactions'].sum().reset_index().head()
bill_detail_test.groupby(['ID'])['credit_line','cash_advance','amount_transactions'].sum().head()
data_bill = bill_detail_test.assign(bill_credit=bill_detail_test.last_bill/bill_detail_test.credit_line).groupby('ID')['bill_credit'].mean().to_frame().replace({np.inf:2}).reset_index().merge(bill_detail_test.groupby(['ID'])['credit_line','cash_advance','amount_transactions'].sum().reset_index(),on='ID',how='outer')

data_bill.head()

data_browser = browse_history_test.groupby(['ID','browser_code']).browse_data.sum().reset_index().pivot(index='ID', columns='browser_code', values='browse_data').fillna(0)

data_browser.columns = ['browser'+ str(x) for x in data_browser.columns]

data_browser.reset_index().head()

Index(['ID', 'timestamp_bill', 'bank_id', 'last_bill', 'last_repayment',
       'credit_line', 'current_balance', 'minimum_payments',
       'amount_transactions', 'current_money', 'adjust_money',
       'cycle_interest', 'available_money', 'cash_advance',
       'repayment_status'],
      dtype='object')
Index(['ID', 'timestamp_bank', 'type', 'money', 'salary', 'money_direction'], dtype='object')


,ID,browser1,browser2,browser3,browser4,browser5,browser6,browser7,browser8,browser9,browser10,browser11
0,55597,23248.0,0.0,0.0,8872.0,560.0,1200.0,5312.0,0.0,0.0,4560.0,0.0
1,55598,222296.0,0.0,0.0,96576.0,30608.0,19304.0,24288.0,23512.0,0.0,33440.0,0.0
2,55601,58104.0,0.0,0.0,28536.0,15320.0,3600.0,4496.0,1336.0,0.0,13680.0,0.0
3,55602,46704.0,0.0,0.0,18264.0,17316.0,2400.0,8424.0,6012.0,0.0,14448.0,0.0
4,55603,6624.0,0.0,0.0,3664.0,3640.0,400.0,1184.0,0.0,0.0,1520.0,0.0


## 数据合并

In [17]:
data_bill.ID = data_bill.ID.astype(str)
data_browser = data_browser.reset_index()
data_browser.ID = data_browser.ID.astype(str)

data_test = data_bank.merge(data_bill,on='ID',how='left').merge(data_browser,on='ID',how='left')

print(data_test.shape)
print(data_test.isnull().sum()/len(data_test))
print(data_test.shape)

data_test.to_csv('/Users/Evan/Desktop/data_test.csv')

(13899, 32)
ID                     0.000000
gender                 0.000000
career                 0.000000
education              0.000000
marriage               0.000000
hukou                  0.000000
timestamp_money        0.000000
Label                  0.000000
freq_total             0.948989
mean_total             0.948989
sum_total              0.948989
freq_income            0.948989
mean_income            0.948989
sum_income             0.948989
freq_spend             0.949277
mean_spend             0.949277
sum_spend              0.949277
bill_credit            0.113605
credit_line            0.018419
cash_advance           0.018419
amount_transactions    0.018419
browser1               0.136844
browser2               0.136844
browser3               0.136844
browser4               0.136844
browser5               0.136844
browser6               0.136844
browser7               0.136844
browser8               0.136844
browser9               0.136844
browser10              0.136